In [190]:
from matplotlib.image import imread
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
from collections import Counter

In [191]:
#resize board image to (640, 640)
def resize_board_image(img_path):
    img = Image.open(img_path)
    img = img.resize((640, 640))
    img.save(img_path)
    return img_path

In [192]:
def grayscale(board):
    #grayscale value
    #RGB Value to grayscale formula (NTSC):
    #0.299 ∙ Red + 0.587 ∙ Green + 0.114 ∙ Blue
    board_gray = []
    for i in range(board.shape[0]):
        board_gray_row = []
        for j in range (board.shape[1]):
            r = board[i,j,0]
            g = board[i,j,1]
            b = board[i,j,2]
            board_gray_row.append(0.299*r + 0.587*g + 0.114*b)
        board_gray.append(board_gray_row)
    return np.array(board_gray)

In [193]:
def get_squares(board):
    squares = []
    square_width = int(board.shape[0]/8)
    square_height = int(board.shape[1]/8)
    for i in range(8):
        squares_row = []
        for j in range (8):
            square = board[(square_height*i):(square_height*(i+1)), (square_width*j):(square_width*(j+1))]
            squares_row.append(square)
        squares.append(squares_row)
    return np.array(squares)

In [194]:
def prewitt(square):
    square_height, square_width = square.shape
    #prewitt kernel
    h_filter = np.array([[-1, 0, 1], 
                        [-1, 0, 1], 
                        [-1, 0, 1]])
    v_filter = np.array([[-1, -1, -1], 
                        [ 0,  0,  0], 
                        [ 1,  1,  1]])

    #initialize gradient image
    gradient_image = np.zeros((square_height, square_width))

    for i in range(1, square_height - 1):
        for j in range(1, square_width - 1):
            h_value = (h_filter[0, 0] * square[i - 1, j - 1]) + \
                      (h_filter[0, 1] * square[i - 1, j]) + \
                      (h_filter[0, 2] * square[i - 1, j + 1]) + \
                      (h_filter[1, 0] * square[i, j - 1]) + \
                      (h_filter[1, 1] * square[i, j]) + \
                      (h_filter[1, 2] * square[i, j + 1]) + \
                      (h_filter[2, 0] * square[i + 1, j - 1]) + \
                      (h_filter[2, 1] * square[i + 1, j]) + \
                      (h_filter[2, 2] * square[i + 1, j + 1])

            v_value = (v_filter[0, 0] * square[i - 1, j - 1]) + \
                      (v_filter[0, 1] * square[i - 1, j]) + \
                      (v_filter[0, 2] * square[i - 1, j + 1]) + \
                      (v_filter[1, 0] * square[i, j - 1]) + \
                      (v_filter[1, 1] * square[i, j]) + \
                      (v_filter[1, 2] * square[i, j + 1]) + \
                      (v_filter[2, 0] * square[i + 1, j - 1]) + \
                      (v_filter[2, 1] * square[i + 1, j]) + \
                      (v_filter[2, 2] * square[i + 1, j + 1])
            #value per pixel in gradient image
            magnitude = np.sqrt(pow(h_value, 2) + pow(v_value, 2))
            gradient_image[i - 1, j - 1] = magnitude
    return gradient_image

In [195]:
#crop out border
def crop(source, value):
    square = []
    for i in range(int(source.shape[0]*value), int(source.shape[0]*(1-value))):
        square_row = []
        for j in range (int(source.shape[0]*value), int(source.shape[0]*(1-value))):
            square_row.append(source[i][j])
        square.append(square_row)
    return np.array(square)

In [196]:
#check square occupancy
def is_occupied(square):
    piece = crop(square, 0.125)
    piece = piece + 0.1e-16 #to avoid dividing by zero
    
    #horizontal pass
    filled_h = 0
    for i in range(piece.shape[0]):
        target = 0
        ratio = 0
        stop_count = 0
        for j in range(piece.shape[1]-1):
            ratio = piece[i, j+1] / piece[i, j] #right:left pixel
            if ratio > 9: #prewitt creates high contrast between pixels on an edge
                target += 1
                stop_count = j+1 #exclude detected pixel pair from the next search
                break
        ratio = 0
        for j in range(piece.shape[1]-1-stop_count):
            ratio = piece[i, piece.shape[1]-j-2] / piece[i, piece.shape[1]-j-1] #left:right pixel
            if ratio > 9:
                target += 1
                break
        if target >= 2:
            filled_h += 1
        else:
            pass
    
    #vertical pass
    filled_v = 0
    for i in range(piece.shape[1]):
        target = 0
        ratio = 0
        stop_count = 0
        for j in range(piece.shape[0]-1):
            ratio = piece[j+1, i] / piece[j, i] #bottom:top pixel
            if ratio > 9: 
                target += 1
                stop_count = j+1
                break
        ratio = 0
        for j in range(piece.shape[0]-1-stop_count):
            ratio = piece[piece.shape[1]-j-2, i] / piece[piece.shape[1]-j-1, i] #top:bottom pixel
            if ratio > 9:
                target += 1
                break
        if target >= 2:
            filled_v += 1
        else:
            pass
#     print(f"Horizontal: {filled_h/piece.shape[0]}", f"Vertical: {filled_v/piece.shape[1]}", piece.shape)
    return filled_h/piece.shape[0] > 0.6 and filled_v/piece.shape[1] > 0.3

In [197]:
#flag occupied squares
def flag_squares(squares): #prewitt input
    flags = []
    for s_row in squares:
        f_row = []
        for square in s_row:
            if is_occupied(square):
                f_row.append(1)
            else:
                f_row.append(0)
        flags.append(f_row)
    return np.array(flags)

In [198]:
#get feature vector
def feature_vector(array): #ndarray input
    return np.reshape(array, array.size)

In [199]:
#euclidean distance
def euclidean_distance(v1, v2):
    if(v1.size != v2.size):
        print("Error")
    squared_differences = 0
    for i in range(v1.size):
        squared_differences += pow((v1[i] - v2[i]), 2)
    return np.sqrt(squared_differences)

In [200]:
#TEST
board = imread(resize_board_image("chessboard_test.png"))
squares = get_squares(prewitt(grayscale(board)))

In [201]:
flag_squares(squares)

array([[1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1]])

In [174]:
a = (np.array([1,1]) + np.array([1,1]))
print(a)
a += np.array([2,2])
print(a)

[2 2]
[4 4]


In [204]:
#TODO
#get data from data_prep
#label data
#test SVM